<img src="download (1).png" />

# I will connect to a coinmarketcap.com public API and pull the data to a snowflake database. i will be gathering data about cryptocurrency and will be doing EDA on it.

# installing & importing needed dependencies

In [ ]:
pip install snowflake-connector-python

In [ ]:
pip install snowflake-sqlalchemy

In [11]:
import requests
import json
import snowflake.connector as sf
import pandas as pd

# To connect to a Snowflake database and create a storage schema, you need to establish the connection using credentials and parameters, and then execute the required commands for schema creation. 

In [ ]:
try:
    conn = sf.connect(
        user='user',
        password='password',
        account='pv92401.europe-west4.gcp',
    )
    print("connected")
except Exception as e:
    print(f"Error connecting to Snowflake: {e}")
    exit()

# Creating a cursor object
cur = conn.cursor()
print("successfully created cursor")

# Creating a new warehouse
cur.execute("CREATE WAREHOUSE IF NOT EXISTS crypto")
print("successfully created warehouse")

# Creating a new database
cur.execute("CREATE DATABASE IF NOT EXISTS api")
print("successfully created database")

# Creating a new schema
cur.execute("USE DATABASE api")
cur.execute("CREATE SCHEMA IF NOT EXISTS eda")
print("successfully created schema")

# Creating a new table
try:
    cur.execute("USE SCHEMA eda")
    cur.execute("CREATE TABLE IF NOT EXISTS coins (name VARCHAR(50), tags VARCHAR(500), marketcap NUMBER(38, 8))")
except Exception as e:
    print(f"Error creating table: {e}")
    cur.close()
    conn.close()
    exit()

# Closing the cursor and connection
cur.close()
conn.close()


# The code retrieves specific data from the CoinMarketCap API, namely the Name, Tags, and Market Cap (market capitalization) of cryptocurrencies.

In [5]:
headers = {
    'X-CMC_PRO_API_KEY': 'api_key',
    'Accepts': 'application/json'
}

# passing arguments to fetch 1st and up to 3rd coin
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
    'start': '1',
    'limit': '2000',
    'convert': 'EUR'
}

# send the request and get the response
try:
    response = requests.get(url, params=parameters, headers=headers).json()
    coins = response['data']
    print("Data retrieved successfully.")
except requests.exceptions.RequestException as e:
    print("Error retrieving data:", e)

conn = sf.connect(
    user='Vandalism',
    password='Rakovosel0123321',
    account='pv92401.europe-west4.gcp',
    warehouse='crypto1',
    database='api1',
    schema='eda1'
)

cur = conn.cursor()

coins = response['data']
print(f"Retrieved {len(coins)} coins.")

#adding jupyter to interpret in dataframe format

for coin in coins:
    name = coin['name']
    tags = coin['tags']
    marketcap = coin['quote']['EUR']['market_cap']
    print(f"Processing data for {name}")
    
    try:
        query = """
            MERGE INTO coins1 c
            USING (SELECT %s name, %s tags, %s marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        """

        print(f"Query: {query % (name, json.dumps(tags), marketcap)}")
        cur.execute(query, (name, json.dumps(tags), marketcap))
        print(f"Data for {name} inserted successfully.")
        
    except sf.Error as e:
        print(f"Error inserting data for {name}:", e)

Data retrieved successfully.
Retrieved 2000 coins.
Processing data for Bitcoin
Query: 
            MERGE INTO coins1 c
            USING (SELECT Bitcoin name, ["mineable", "pow", "sha-256", "store-of-value", "state-channel", "coinbase-ventures-portfolio", "three-arrows-capital-portfolio", "polychain-capital-portfolio", "binance-labs-portfolio", "blockchain-capital-portfolio", "boostvc-portfolio", "cms-holdings-portfolio", "dcg-portfolio", "dragonfly-capital-portfolio", "electric-capital-portfolio", "fabric-ventures-portfolio", "framework-ventures-portfolio", "galaxy-digital-portfolio", "huobi-capital-portfolio", "alameda-research-portfolio", "a16z-portfolio", "1confirmation-portfolio", "winklevoss-capital-portfolio", "usv-portfolio", "placeholder-ventures-portfolio", "pantera-capital-portfolio", "multicoin-capital-portfolio", "paradigm-portfolio", "btc-ecosystem"] tags, 519866229348.1538 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SE

Error inserting data for Polkadot: 100078 (22000): String '["substrate", "polkadot", "binance-chain", "polkadot-ecosystem", "three-arrows-capital-portfolio", "polychain-capital-portfolio", "arrington-xrp-capital-portfolio", "blockchain-capital-portfolio", "bo...' is too long and would be truncated
Processing data for Litecoin
Query: 
            MERGE INTO coins1 c
            USING (SELECT Litecoin name, ["mineable", "pow", "scrypt", "medium-of-exchange"] tags, 5900580644.156476 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Litecoin inserted successfully.
Processing data for TRON
Query: 
            MERGE INTO coins1 c
            USING (SELECT TRON name, ["media", "payments", "tron-ecosystem"] tags, 5859122104.446742 marketcap) v
            ON c.name = 

Data for OKB inserted successfully.
Processing data for Ethereum Classic
Query: 
            MERGE INTO coins1 c
            USING (SELECT Ethereum Classic name, ["mineable", "pow", "ethash", "platform", "smart-contracts", "dcg-portfolio"] tags, 2510594476.1483164 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Ethereum Classic inserted successfully.
Processing data for Internet Computer
Query: 
            MERGE INTO coins1 c
            USING (SELECT Internet Computer name, ["platform", "distributed-computing", "collectibles-nfts", "defi", "interoperability", "smart-contracts", "dao", "wallet", "storage", "polychain-capital-portfolio", "lending-borowing", "metaverse", "electric-capital-portfolio", "fenbushi-capital-portfolio", "hashkey-capital-portfolio", 

Data for VeChain inserted successfully.
Processing data for Pepe
Query: 
            MERGE INTO coins1 c
            USING (SELECT Pepe name, ["memes"] tags, 1348821245.283226 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Pepe inserted successfully.
Processing data for ApeCoin
Query: 
            MERGE INTO coins1 c
            USING (SELECT ApeCoin name, ["collectibles-nfts", "content-creation", "gaming", "entertainment", "dao", "governance", "metaverse", "animoca-brands-portfolio"] tags, 1280812383.7769096 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.m

Data for Rocket Pool inserted successfully.
Processing data for Tezos
Query: 
            MERGE INTO coins1 c
            USING (SELECT Tezos name, ["pos", "platform", "enterprise-solutions", "collectibles-nfts", "defi", "smart-contracts", "polychain-capital-portfolio", "boostvc-portfolio", "winklevoss-capital-portfolio", "layer-1"] tags, 870872635.4323443 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Tezos inserted successfully.
Processing data for BitDAO
Query: 
            MERGE INTO coins1 c
            USING (SELECT BitDAO name, ["dao", "governance", "dragonfly-capital-portfolio", "fenbushi-capital-portfolio", "pantera-capital-portfolio", "spartan-group"] tags, 862041426.3310624 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN

...

Data for DinoSwap inserted successfully.
Processing data for Tokenplace
Query: 
            MERGE INTO coins1 c
            USING (SELECT Tokenplace name, [] tags, 207743.40059102824 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Tokenplace inserted successfully.
Processing data for Blue Protocol
Query: 
            MERGE INTO coins1 c
            USING (SELECT Blue Protocol name, ["platform"] tags, 207679.84319981321 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Blue Protocol inserted successfully.
Processing data for Terracoi

Data for MediShares inserted successfully.
Processing data for BiblePay
Query: 
            MERGE INTO coins1 c
            USING (SELECT BiblePay name, ["mineable", "philanthropy", "masternodes", "payments"] tags, 192010.66870672157 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for BiblePay inserted successfully.
Processing data for ASPO World
Query: 
            MERGE INTO coins1 c
            USING (SELECT ASPO World name, ["gaming", "polkafoundry-red-kite", "play-to-earn", "icetea-labs"] tags, 191671.80046488158 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags,

Data for Lead Wallet inserted successfully.
Processing data for RED
Query: 
            MERGE INTO coins1 c
            USING (SELECT RED name, ["marketplace"] tags, 186543.69105446222 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for RED inserted successfully.
Processing data for Golff
Query: 
            MERGE INTO coins1 c
            USING (SELECT Golff name, ["yield-aggregator", "heco-ecosystem"] tags, 186088.0079411579 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Golff inserted successfully.
Processing data for Creo Engine


Data for PWR Coin inserted successfully.
Processing data for Motocoin
Query: 
            MERGE INTO coins1 c
            USING (SELECT Motocoin name, ["mineable", "pop"] tags, 174113.80698131616 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Motocoin inserted successfully.
Processing data for Phore
Query: 
            MERGE INTO coins1 c
            USING (SELECT Phore name, ["privacy", "masternodes"] tags, 173905.20131788144 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for Phore inserted successfully.
Processing data for WeTrust

Data for NFTify inserted successfully.
Processing data for U Network
Query: 
            MERGE INTO coins1 c
            USING (SELECT U Network name, ["media", "content-creation", "smart-contracts"] tags, 165923.77182567446 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Data for U Network inserted successfully.
Processing data for Team Vitality Fan Token
Query: 
            MERGE INTO coins1 c
            USING (SELECT Team Vitality Fan Token name, ["sports", "fan-token"] tags, 165481.4539787255 marketcap) v
            ON c.name = v.name
            WHEN MATCHED THEN
                UPDATE SET c.tags = v.tags, c.marketcap = v.marketcap
            WHEN NOT MATCHED THEN
                INSERT (name, tags, marketcap) VALUES (v.name, v.tags, v.marketcap)
        
Dat

# bitcoin & ethereum had too many characthers in tags column to be truncated to i altered the table and manually put in the tags

In [19]:
conn = sf.connect(
    user='Vandalism',
    password='Rakovosel0123321',
    account='pv92401.europe-west4.gcp',
    warehouse='crypto',
    database='api',
    schema='eda'
)
print("connected")
# Create a cursor object to execute SQL queries
cur = conn.cursor()
print("created cursor")

sql = """ALTER TABLE coins2 MODIFY COLUMN tags VARCHAR(2000);"""

# Execute the query
cur.execute(sql)

# Construct the query to update the tags of Bitcoin in coins2 table
sql = """UPDATE coins2
         SET tags = CONCAT(tags, ',"pow","sha-256","mineable","store-of-value","state-channel","coinbase-ventures-portfolio","three-arrows-capital-portfolio","polychain-capital-portfolio","binance-labs-portfolio","blockchain-capital-portfolio","boostvc-portfolio","cms-holdings-portfolio","dcg-portfolio","dragonfly-capital-portfolio","electric-capital-portfolio","fabric-ventures-portfolio","framework-ventures-portfolio","galaxy-digital-portfolio","huobi-capital-portfolio","alameda-research-portfolio","a16z-portfolio","1confirmation-portfolio","winklevoss-capital-portfolio","usv-portfolio","placeholder-ventures-portfolio","pantera-capital-portfolio","multicoin-capital-portfolio","paradigm-portfolio","pos","smart-contracts","ethereum-ecosystem","hashkey-capital-portfolio","kenetic-capital-portfolio","injective-ecosystem","payments","stablecoin","asset-backed-stablecoin","avalanche-ecosystem","solana-ecosystem","arbitrum-ecosytem","moonriver-ecosystem","bnb-chain","usd-stablecoin","optimism-ecosystem"')
         WHERE name = 'Bitcoin'"""

cur.execute(sql)

sql = """UPDATE coins2
         SET tags = CONCAT(tags, ', "PoS", "Smart Contracts", "Coinbase Ventures Portfolio", "Three Arrows Capital Portfolio", "Polychain Capital Portfolio", "Binance Labs Portfolio", "Blockchain Capital Portfolio", "BoostVC Portfolio", "CMS Holdings Portfolio", "DCG Portfolio", "DragonFly Capital Portfolio", "Electric Capital Portfolio", "Fabric Ventures Portfolio", "Framework Ventures Portfolio", "Hashkey Capital Portfolio", "Kenetic Capital Portfolio", "Huobi Capital Portfolio", "Alameda Research Portfolio", "a16z Portfolio", "1Confirmation Portfolio", "Winklevoss Capital Portfolio", "USV Portfolio", "Placeholder Ventures Portfolio", "Pantera Capital Portfolio", "Multicoin Capital Portfolio", "Paradigm Portfolio", "Ethereum Ecosystem", "Injective Ecosystem"')
         WHERE name = 'Ethereum'"""

# Execute the query
cur.execute(sql)

# Commit the changes to the database
conn.commit()

# Print a message indicating the update was successful
print("Updated")

connected
created cursor
Updated


In [28]:
conn = sf.connect(
    user='Vandalism',
    password='Rakovosel0123321',
    account='pv92401.europe-west4.gcp',
    warehouse='crypto1',
    database='api1',
    schema='eda1'
)
print("connected")
# Create a cursor object to execute SQL queries
cur = conn.cursor()
print("created cursor")


cur.execute("SELECT * FROM coins1 ORDER BY marketcap DESC")

df = pd.DataFrame(cur.fetchall(), columns= ['name','tags','marketcap'])
conn.close()

connected
created cursor
